In [2]:
#Train, val, Test splitter
import os
from sklearn.model_selection import train_test_split
import shutil
import random
import textgrid as txt
import pandas as pd

In [3]:
path = "raw_data/anno/annotation-EN-lppEN_section1.TextGrid"
file = txt.TextGrid.fromFile(path)

In [4]:
#rejects words that happen in two/in between two scans
result = []
tmp = []
for _, i in enumerate(range(len(file[0]))):
    j = file[0][i].maxTime
    if (j-len(result)*2) < 2:
        tmp.append(file[0][i].mark)
    else:
        result.append(tmp)
        tmp = []
result

[['#', 'once', 'when', 'i', 'was', 'six', 'years', 'old'],
 ['i', 'saw', 'a', 'magnificent'],
 ['in', 'a', 'book', 'about', 'the', 'primeval'],
 ['called', 'real', 'life'],
 ['#', 'it', 'showed', 'a'],
 ['constrictor', 'swallowing', 'a', 'wild'],
 ['#', 'here', 'is', 'a', 'copy', 'of'],
 ['drawing', '#'],
 ['said', 'in', 'the', 'book', '#', 'boa'],
 ['swallow', 'their', 'prey', 'whole'],
 ['chewing', '#', 'then'],
 ['are', 'not', 'able', 'to', 'move', 'and', 'they'],
 ['for', 'the', 'six', 'months', 'it', 'takes', 'for'],
 ['#', 'so', 'i'],
 ['a', 'lot', 'about', 'the', 'adventures', 'of', 'the'],
 ['#', 'and', 'in', 'turn', 'i'],
 ['with', 'a', 'coloured', 'pencil', '#', 'to', 'make'],
 ['first', 'drawing', '#', 'my', 'drawing'],
 ['one', '#', 'it', 'looked', 'like'],
 ['#', 'i'],
 ['my', 'masterpiece', 'to', 'the', 'grownups'],
 ['i', 'asked', 'them', 'if', 'my', 'drawing', 'frightened'],
 ['#', 'they', 'answered', 'me', '#', 'why'],
 ['anyone', 'be', 'frightened', 'by', 'a'],
 ['#',

In [5]:
# adds words that happen in transistion between scans
# to the first scan it appears in but not the second
result = []
tmp = []
for _, i in enumerate(range(len(file[0]))):
    j = file[0][i].maxTime
    if (j-len(result)*2) <= 2:
        tmp.append(file[0][i].mark)
    else:
        if file[0][i].minTime-len(result)*2 <= 2:
            tmp.append(file[0][i].mark)
        result.append(tmp)
        tmp = []
result


[['#', 'once', 'when', 'i', 'was', 'six', 'years', 'old', '#'],
 ['i', 'saw', 'a', 'magnificent', 'picture'],
 ['in', 'a', 'book', 'about', 'the', 'primeval', 'forest'],
 ['called', 'real', 'life', 'stories'],
 ['#', 'it', 'showed', 'a', 'boa'],
 ['constrictor', 'swallowing', 'a', 'wild', 'animal'],
 ['#', 'here', 'is', 'a', 'copy', 'of', 'the'],
 ['drawing', '#', 'it'],
 ['said', 'in', 'the', 'book', '#', 'boa', 'constrictors'],
 ['swallow', 'their', 'prey', 'whole', 'without'],
 ['chewing', '#', 'then', 'they'],
 ['are', 'not', 'able', 'to', 'move', 'and', 'they', 'sleep'],
 ['for', 'the', 'six', 'months', 'it', 'takes', 'for', 'digestion'],
 ['#', 'so', 'i', 'thought'],
 ['a', 'lot', 'about', 'the', 'adventures', 'of', 'the', 'jungle'],
 ['#', 'and', 'in', 'turn', 'i', 'managed'],
 ['with', 'a', 'coloured', 'pencil', '#', 'to', 'make', 'my'],
 ['first', 'drawing', '#', 'my', 'drawing', 'number'],
 ['one', '#', 'it', 'looked', 'like', 'this'],
 ['#', 'i', 'showed'],
 ['my', 'masterpi

In [6]:
# adds words that happen in transistion between scans
# to both scans it appears in
result = []
tmp = []
for _, i in enumerate(range(len(file[0]))):
    j = file[0][i].maxTime
    if (j-len(result)*2) <= 2:
        tmp.append(file[0][i].mark)
    else:
        if file[0][i].minTime-len(result)*2 <= 2:
            o = file[0][i].mark
            tmp.append(o)
        result.append(tmp)
        tmp = [o]
result


[['#', 'once', 'when', 'i', 'was', 'six', 'years', 'old', '#'],
 ['#', 'i', 'saw', 'a', 'magnificent', 'picture'],
 ['picture', 'in', 'a', 'book', 'about', 'the', 'primeval', 'forest'],
 ['forest', 'called', 'real', 'life', 'stories'],
 ['stories', '#', 'it', 'showed', 'a', 'boa'],
 ['boa', 'constrictor', 'swallowing', 'a', 'wild', 'animal'],
 ['animal', '#', 'here', 'is', 'a', 'copy', 'of', 'the'],
 ['the', 'drawing', '#', 'it'],
 ['it', 'said', 'in', 'the', 'book', '#', 'boa', 'constrictors'],
 ['constrictors', 'swallow', 'their', 'prey', 'whole', 'without'],
 ['without', 'chewing', '#', 'then', 'they'],
 ['they', 'are', 'not', 'able', 'to', 'move', 'and', 'they', 'sleep'],
 ['sleep', 'for', 'the', 'six', 'months', 'it', 'takes', 'for', 'digestion'],
 ['digestion', '#', 'so', 'i', 'thought'],
 ['thought', 'a', 'lot', 'about', 'the', 'adventures', 'of', 'the', 'jungle'],
 ['jungle', '#', 'and', 'in', 'turn', 'i', 'managed'],
 ['managed', 'with', 'a', 'coloured', 'pencil', '#', 'to', '

In [8]:
path = "raw_data/anno/annotation-EN-lppEN_word_information.csv"
file = pd.read_csv(path)
# adds words that happen in transistion between scans
# to both scans it appears in
result = {k:[] for k in range(1,10)}
tmp = []
count = 0
sec = 1
for idx, row in file.iterrows():
    if row["section"] != sec:
        count = 0
        tmp = []
    sec = row["section"]
    if True: #row["pos"] == "VERB":
        w = row["lemma"]
        if (row["onset"]-count*2) < 2:# and row["pos"] == "VERB":
            tmp.append(w)
        if (row["offset"]-count*2) > 2:
            result[row["section"]].append((tmp))
            count +=1 
            if (row["offset"]-count*2) < 2:## and row["pos"] == "VERB":
                tmp = [w]
            else:
                tmp = []
    else:
        result.append(tmp)
        tmp = []
result


{1: [['once', 'when', 'i', 'was', 'six', 'years', 'old'],
  ['i', 'saw', 'a', 'magnificent', 'picture'],
  ['picture', 'in', 'a', 'book', 'about', 'the', 'primeval', 'forest'],
  ['forest', 'called', 'real', 'life', 'stories'],
  ['stories', 'it', 'showed', 'a', 'boa'],
  ['boa', 'constrictor', 'swallowing', 'a', 'wild'],
  ['animal', 'here', 'is', 'a', 'copy', 'of', 'the'],
  ['the', 'drawing', 'it'],
  ['it', 'said', 'in', 'the', 'book', 'boa', 'constrictors'],
  ['constrictors', 'swallow', 'their', 'prey', 'whole', 'without'],
  ['without', 'chewing', 'then', 'they'],
  ['they', 'are', 'not', 'able', 'to', 'move', 'and', 'they', 'sleep'],
  ['sleep', 'for', 'the', 'six', 'months', 'it', 'takes', 'for', 'digestion'],
  ['digestion', 'so', 'i', 'thought'],
  ['thought', 'a', 'lot', 'about', 'the', 'adventures', 'of', 'the', 'jungle'],
  ['jungle', 'and', 'in', 'turn', 'i', 'managed'],
  ['managed', 'with', 'a', 'colored', 'pencil', 'to', 'make', 'my'],
  ['my', 'first', 'drawing', 'my

In [33]:
path = "raw_data/anno/annotation-EN-lppEN_word_information.csv"
file = pd.read_csv(path)
# adds words that happen in transistion between scans
# to both scans it appears in
result = {k:[] for k in range(9)}
count = 0
sec = 1
for _, row in file.iterrows():
    if row["section"] != sec:
        count = 0
    sec = row["section"]
    w = row["lemma"]
    if (row["onset"]-count*2) < 2:
        count +=1  
        if row["pos"] == "PRON":
            result[sec-1].append(w)
        else:
            result[sec-1].append("")
        

    
result


{0: ['',
  '',
  'i',
  '',
  '',
  '',
  '',
  'i',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'it',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'it',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'they',
  '',
  '',
  '',
  '',
  '',
  '',
  'they',
  '',
  '',
  '',
  '',
  '',
  'it',
  '',
  '',
  '',
  '',
  'i',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'i',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'it',
  '',
  '',
  '',
  'i',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'i',
  '',
  'them',
  '',
  '',
  '',
  '',
  'them',
  'they',
  '',
  'me',
  '',
  '',
  'anyone',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'it',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'i',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''

In [41]:
o = result.values()
oo = [l for s in o for l in s]
set(oo)

{'',
 'anyone',
 'anything',
 'everybody',
 'everyone',
 'everything',
 'he',
 'her',
 'herself',
 'him',
 'himself',
 'his',
 'i',
 'it',
 'itself',
 'me',
 'myself',
 'nobody',
 'nothing',
 'one',
 'oneself',
 's',
 'she',
 'somebody',
 'someone',
 'something',
 'them',
 'themselves',
 'there',
 'they',
 'us',
 'we',
 'what',
 'who',
 'whoever',
 'whom',
 'you',
 'yours',
 'yourself'}

In [44]:
path = "raw_data/anno/annotation-EN-lppEN_word_information.csv"
file = pd.read_csv(path)
target_words = {"me" : 0, "you":1, "myself": 0}
result = {k:[] for k in range(9)}
count = 0
sec = 1
for _, row in file.iterrows():
    if row["section"] != sec:
        count = 0
    sec = row["section"]
    w = row["lemma"]
    if (row["onset"]-count*2) < 2:
        count +=1  
        if w in target_words.values():
            result[sec-1].append(target_words[w])
        else:
            result[sec-1].append("")
        

    
result


{0: ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'me',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',

In [61]:
for f in result.values():
    i = dict(enumerate(set(f)))

In [55]:
file = pd.read_csv(path)
# adds words that happen in transistion between scans
# to both scans it appears in
result = {k:[] for k in range(9)}
count = 0
sec = 1
for _, row in file.iterrows():
    if row["section"] != sec:
        count = 0
    sec = row["section"]
    w = row["lemma"]
    if (row["onset"]-count*2) < 2:
        count +=1  
        if row["pos"] == "PRON":
            result[sec-1].append(w)
        else:
            result[sec-1].append("")
label_set = {v:k for k,v in enumerate(set(result.values()))}
output = {k:[] for k in range(9)}
for run, v in result.items():
    output[run].append(label_set[v])

TypeError: unhashable type: 'list'